# or tools

## 라이브러리

In [12]:
!pip install ortools

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 모델링

In [35]:
import math
import pandas as pd
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
import matplotlib.pyplot as plt
import numpy as np

# 데이터 로드
data_df = pd.read_csv('/content/drive/MyDrive/산타/data/data.csv')
data_df = data_df.sort_values(by='point_id').reset_index(drop=True)

In [36]:
# 출발지(창고)와 마을 데이터 구분
# "DEPOT"이라는 point_id를 가진 행을 출발지로 가정
depot_data = data_df[data_df['point_id'] == 'DEPOT'].iloc[0]
towns_data = data_df[data_df['point_id'] != 'DEPOT'].reset_index(drop=True)

# OR-Tools에 사용할 데이터 준비
# 출발지 좌표를 맨 앞에 두고 이후 마을 좌표들을 나열
locations = [(depot_data['x'], depot_data['y'])] + list(zip(towns_data['x'], towns_data['y']))
demands = [0] + list(towns_data['demand'])  # 출발지(창고)는 수요량(demand)이 0

In [37]:
num_locations = len(locations)
depot_index = 0  # 출발지 인덱스(0)

# 유클리드 거리 함수를 정의
def euclidean_distance(p1, p2):
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

# 거리 행렬(distance matrix) 생성
distance_matrix = [[0]*num_locations for _ in range(num_locations)]
for i in range(num_locations):
    for j in range(num_locations):
        if i != j:
            distance_matrix[i][j] = euclidean_distance(locations[i], locations[j])
        else:
            distance_matrix[i][j] = 0

In [38]:
# VRP(차량경로문제)용 데이터 구조 정의
data = {}
data['distance_matrix'] = distance_matrix
data['demands'] = demands
data['vehicle_capacities'] = [25] * 75  # 차량 최대 75대, 각 차량 용량(capacity) 25로 가정
data['num_vehicles'] = 75  # 차량 수
data['depot'] = depot_index  # 출발지 인덱스

# OR-Tools의 RoutingIndexManager 및 RoutingModel 생성
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)

In [39]:
# 거리 콜백 함수 정의: 특정 노드에서 다른 노드로 이동하는 데 필요한 거리 반환
def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)  # 인덱스를 실제 노드로 변환
    to_node = manager.IndexToNode(to_index)
    return int(data['distance_matrix'][from_node][to_node])

# 콜백 함수를 라우팅 모델에 등록 및 차량 간선 비용(거리) 평가자로 설정
transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [40]:
# 수요(demand) 콜백 함수 정의: 각 노드가 요구하는 수요량 반환
def demand_callback(from_index):
    from_node = manager.IndexToNode(from_index)
    return data['demands'][from_node]

# 수요 콜백 등록
demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)

In [41]:
# 차량 용량 제약 추가
routing.AddDimensionWithVehicleCapacity(
    demand_callback_index,
    0,  # 여유(slack) 없음
    data['vehicle_capacities'],  # 각 차량의 용량 리스트
    True,  # 용량 누적을 0부터 시작
    'Capacity')

# 탐색 파라미터 설정
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION
search_parameters.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.TABU_SEARCH
search_parameters.time_limit.seconds = 60 * 10 # 60 = 1분,
search_parameters.log_search = True  # 탐색 과정 로그 출력

In [42]:
solution = routing.SolveWithParameters(search_parameters)

if solution:
    total_distance = 0
    all_routes = []

    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        route = []
        route_distance = 0
        previous_node = manager.IndexToNode(index)  # DEPOT

        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            route.append(node)
            index = solution.Value(routing.NextVar(index))
            current_node = manager.IndexToNode(index) if not routing.IsEnd(index) else 0  # 다음 노드 또는 DEPOT
            # 이전 노드와 현재 노드 간 거리 추가
            route_distance += data['distance_matrix'][previous_node][current_node]
            previous_node = current_node

        # DEPOT으로 복귀하는 거리 추가
        route_distance += data['distance_matrix'][previous_node][0]

        # 경로가 비어있지 않은 경우만 저장
        if len(route) > 1:
            route.append(0)  # DEPOT으로 복귀
            all_routes.append(route)
            total_distance += route_distance

    # (1) 총 거리 출력
    print("Solution found!")
    print(f"Total Distance: {total_distance:.4f}")

    # 라우트별로 point_id 변환 + 출력
    total_route_town_ids = []
    for i, route in enumerate(all_routes):
        route_town_ids = []
        for node_idx in route:
            if node_idx == 0:
                route_town_ids.append('DEPOT')
            else:
                town_id = towns_data.iloc[node_idx - 1]['point_id']
                route_town_ids.append(town_id)
        print(f"Route #{i + 1}:", " -> ".join(route_town_ids))
        total_route_town_ids += route_town_ids
        total_route_town_ids = total_route_town_ids[:-1]
    total_route_town_ids.append('DEPOT')  # 마지막 DEPOT

    # (2) 시각화
    plt.figure(figsize=(10, 10))
    plt.title(f'Routes Visualization (Total Distance: {round(total_distance, 4)})')

    # 마을(blue) & 창고(red)
    town_x = [loc[0] for loc in locations[1:]]
    town_y = [loc[1] for loc in locations[1:]]
    plt.scatter(town_x, town_y, c='blue', label='Towns', alpha=0.6)
    depot_x, depot_y = locations[0]
    plt.scatter(depot_x, depot_y, c='red', label='Depot', s=100, marker='X')

    # 라우트별로 다른 색상으로 표시
    colors = plt.cm.get_cmap('rainbow', len(all_routes))

    for i, route in enumerate(all_routes):
        route_coords = [locations[node_idx] for node_idx in route]
        route_x = [coord[0] for coord in route_coords]
        route_y = [coord[1] for coord in route_coords]
        plt.plot(route_x, route_y, color=colors(i), label=f'Route {i + 1}', alpha=0.7)

    plt.legend()
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.grid(True)
    plt.show()

else:
    print("No solution found.")


Initial Total Distance: 2235.8074094765857


## 후처리

## 파일 제출

In [34]:
# Prepare submission in the required format
submission_route = []
total_distance = 0

for route in optimized_routes:
    route_distance = 0
    previous_node = 0  # Start from DEPOT (index 0)
    for node_idx in route:
        # Convert node indices to point IDs
        if node_idx == 0:
            submission_route.append('DEPOT')
        else:
            town_id = towns_data.iloc[node_idx - 1]['point_id']
            submission_route.append(town_id)
        # Calculate distance
        route_distance += data['distance_matrix'][previous_node][node_idx]
        previous_node = node_idx
    # Return to DEPOT
    route_distance += data['distance_matrix'][previous_node][0]
    submission_route.append('DEPOT')
    total_distance += route_distance

# Remove duplicate consecutive DEPOT entries
submission_route = [submission_route[i] for i in range(len(submission_route)) if i == 0 or submission_route[i] != submission_route[i-1]]

# Create a DataFrame and save it
submission_df = pd.DataFrame({"point_id": submission_route})
submission_file_path = '/content/drive/MyDrive/산타/data/santa_optimized.csv'
submission_df.to_csv(submission_file_path, index=False)

# Download the optimized submission file
from google.colab import files
files.download('/content/drive/MyDrive/산타/data/santa_optimized.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>